# PyTorch
In this exercise, we will look at some basic functionality of PyTorch. Your are free to use other DL frameworks for your exercises and your project. However, the master solutions and code examples will be in PyTorch.

The [PyTorch documentation](https://pytorch.org/docs/stable/index.html) offers information on its functionality. A lot of the time, your specific question will also have been asked on the [PyTorch Forum](https://discuss.pytorch.org/), often with competent answers by the core developers (Google will find the relevant thread for you).

First, we have to install PyTorch. We will install the basic version for this exercise. For your project, if you want to run on a GPU, you'll have to make sure to have a PyTorch version installed that is compatible with the CUDA version of your NVIDIA drivers. PyTorch has an [installation guide](https://pytorch.org/get-started/locally/) that will help you with getting the right version.

In [1]:
# %pip install -U numpy
# %pip install ipywidgets 
# %pip install torch --index-url https://download.pytorch.org/whl/cu117

In [1]:
import torch

torch.cuda.is_available()

True

## Tensor operations
Most of PyTorch's operations have the same name as in NumPy. The basic object for storing data is the `torch.tensor`, the equivalent of the `np.array`. With the help of the [Tensor tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html), do the following:

- Create a `torch.tensor` with the elements `[[1, 2], [3, 4]]`
- Create a tensor of ones/zeros with the same shape and dtype
- Create a random tensor of the same shape
- Print the tensor's shape, data type and device
- Try to move it to the GPU
- For Mac users: Try to move it to [MPS](https://pytorch.org/docs/stable/notes/mps.html)
- Check out indexing/slicing operations, and how you can assign values to a slice.
- Combine tensors with `torch.cat` and `torch.stack`. What are the differences?
- Multiply tensors, element-wise and with matrix multiplication.

In [3]:
x = torch.Tensor([[1, 2], [3, 4]])

x_ones = torch.ones_like(x, dtype=x.dtype)
x_zeroes = torch.zeros_like(x, dtype=x.dtype)
x_random = torch.rand_like(x, dtype=x.dtype)

print(f"Shape of tensor: {x.shape}")
print(f"Datatype of tensor: {x.dtype}")
print(f"Device tensor is stored on: {x.device}")

x_cuda = x.to("cuda")
print(f"Device of cuda tensor: {x_cuda.device}")

x[:,1] = 0
print(x)

x[1:] = 5
print(x)

# torch.stack: Creates a new dimension while stacking the tensors, the input tensors must have the same shape, returns a new tensor with one more dimension than the input tensors
# torch.cat: concatenates tensors along an existing dimension, the tensors must have the same shape except in the dimension along which they are concatenated, does not add a new dimension.
print("torch.cat with dim=1")
t1 = torch.cat([x, x, x], dim=1)
print(t1)

print("torch.cat with dim=0")
t2 = torch.cat([x, x, x], dim=0)
print(t2)

print("torch.stack")
t3 = torch.stack([x, x, x], dim=1)
print(t3)

Shape of tensor: torch.Size([2, 2])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu
Device of cuda tensor: cuda:0
tensor([[1., 0.],
        [3., 0.]])
tensor([[1., 0.],
        [5., 5.]])
torch.cat with dim=1
tensor([[1., 0., 1., 0., 1., 0.],
        [5., 5., 5., 5., 5., 5.]])
torch.cat with dim=0
tensor([[1., 0.],
        [5., 5.],
        [1., 0.],
        [5., 5.],
        [1., 0.],
        [5., 5.]])
torch.stack
tensor([[[1., 0.],
         [1., 0.],
         [1., 0.]],

        [[5., 5.],
         [5., 5.],
         [5., 5.]]])


## Neural Network Basics
Solve the followings tasks with the help of the [Neural networks tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html).

The `nn.Module` is the basic class for layers, networks and models. All parameters of an `nn.Module` are automatically discovered by PyTorch and updated by back-propagation.

First, define a neural network (as a subclass of `nn.Module`) with two linear layers and a ReLU non-linearity in between. Make the input, output, and inner dimensions parameters of your network.

In [5]:
import torch.nn as nn

In [10]:
class Net(nn.Module):
  def __init__(self, input_dim, inner_dim, output_dim):
    super(Net, self).__init__()

    self.relu = nn.ReLU()

    self.fc1 = nn.Linear(input_dim, inner_dim)
    self.fc2 = nn.Linear(inner_dim, output_dim)

  def forward(self, input):
    out = self.relu(self.fc1(input))
    out = self.fc2(out)

    return out

In [15]:
input_dim = 10
inner_dim = 20
output_dim  = 5

model = Net(input_dim=input_dim, inner_dim=inner_dim, output_dim=output_dim)
print(model)

Net(
  (relu): ReLU()
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=5, bias=True)
)


Move the entire network to the GPU/MPS.

In [16]:
if torch.cuda.is_available():
  model = model.to("cuda")
else:
  print("CUDA is not available!")

Print the parameters of your network.

In [18]:
params = list(model.parameters())
print(len(params))
for name, params in model.named_parameters():
  print(name, params.shape)

4
fc1.weight torch.Size([20, 10])
fc1.bias torch.Size([20])
fc2.weight torch.Size([5, 20])
fc2.bias torch.Size([5])


Run a single forward-pass with a random input.

In [23]:
batch_size = 3

input = torch.randn(batch_size, input_dim).to("cuda")
out = model(input)

print(out.shape)
print(out)

torch.Size([3, 5])
tensor([[-0.2815, -0.3024, -0.1247, -0.0754,  0.2216],
        [-0.4789, -0.2851,  0.0423, -0.0122,  0.1430],
        [-0.6411, -0.3181, -0.0855, -0.2721, -0.0074]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


Define a `nn.MSELoss` and a random target.

In [24]:
criterion = nn.MSELoss() # Mean Squared Error
target = torch.randn(batch_size, output_dim, device="cuda")

Compute the loss and run backpropagation.

In [26]:
loss = criterion(out, target)
print(loss)
model.zero_grad()
loss.backward()

tensor(0.6147, device='cuda:0', grad_fn=<MseLossBackward0>)


Update the parameters of your network with a learning rate of 0.01.

In [27]:
learning_rate = 0.01

for param in model.parameters():
    param.data = param.data - (param.grad.data * learning_rate)

Use the `AdamOptimizer` instead to update your parameters (see the [torch.optim documentation](https://pytorch.org/docs/stable/optim.html)).

In [28]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

output = model(input)
loss = criterion(output, target)

optimizer.zero_grad()
loss.backward()
optimizer.step() # updates the model's parameters